# ESGF CWT subset example

taken from esgf-api example:
https://github.com/ESGF/esgf-compute-api/blob/master/examples/subset.ipynb    

In [ ]:
from owslib.wps import WebProcessingService, ComplexDataInput

Get a new token from: https://aims2.llnl.gov/

Login with your ESGF OpendID (LLNL works).

In [ ]:
# token
api_key = 'rj0AgWZtNAuAlBOkfaxEibdNSr4rnCkVUwhFj4PQdZAFddkXlSkakLqQKgCtq6fa'


In [ ]:
# api_key should be in the HTTP headers
client = WebProcessingService('https://aims2.llnl.gov/wps/', verify=True, headers={'COMPUTE-TOKEN': api_key})

### GetCapabilities
https://aims2.llnl.gov/wps/?service=WPS&version=1.0.0&request=GetCapabilities

In [ ]:
for p in client.processes:
    print(p.identifier)

### DescribeProcess
https://aims2.llnl.gov/wps/?service=WPS&version=1.0.0&request=DescribeProcess&identifier=CDAT.subset

In [ ]:
proc = client.describeprocess(
    'CDAT.subset'
)
proc.identifier

In [ ]:
for inpt in proc.dataInputs:
    print(inpt.identifier, inpt.dataType)

### Domain

In [ ]:
from owslib_esgfwps import Domain, Dimension

In [ ]:
d0 = Domain(dict(
        time=Dimension(0, 2000, crs='values'),
        lat=Dimension(-90, 0, crs='values'),
))
print(d0.json)

### Variable

In [ ]:
from owslib_esgfwps import Variable

In [ ]:
# (122, 240, 480)
data_url = 'http://aims3.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r10i1p1f1/Amon/tas/gn/v20180830/tas_Amon_GISS-E2-1-G_historical_r10i1p1f1_gn_185001-190012.nc'

In [ ]:
v0 = Variable(uri=data_url, var_name='tas')
print(v0.json)

### Operation

In [ ]:
from owslib_esgfwps import Operation
op = Operation(name=proc.identifier, domain=d0, input=[v0])
op.json

In [ ]:
from owslib_esgfwps import Variables, Domains, Operations
inputs = []
inputs.append(('domain', Domains([d0])))
inputs.append(('variable', Variables([v0])))
inputs.append(('operation', Operations([op])))

In [ ]:
for inp in inputs:
    print(inp[1])

### Execute
https://aims2.llnl.gov/wps/?service=WPS&version=1.0.0&request=Execute&identifier=CDAT.subset&api_key=TOKEN&DataInputs=domain={};variable={};operation={}

In [ ]:
from owslib.wps import SYNC, ASYNC
exec = client.execute(proc.identifier, inputs=inputs, mode=ASYNC)
print(exec.statusLocation)


In [ ]:
# TODO sync option is not working

from owslib.wps import monitorExecution
monitorExecution(exec)

In [ ]:
print("status: {}, progress: {}/100, message: {}".format(exec.status, exec.percentCompleted, exec.statusMessage))

In [ ]:
for output in exec.processOutputs:
    print(output.identifier, output.reference or output.data)

### Show Output

In [ ]:
from owslib_esgfwps import Outputs

In [ ]:
# *TODO*: returns a list of str instead of dict

outputs = Outputs.from_owslib(exec.processOutputs)